In [2]:
import json
json.loads('[{"variant_id":332,"rating":"none"}]')

[{'variant_id': 332, 'rating': 'none'}]

In [3]:
ratings = [{"variant_id":332,"rating":"none"}]
stringify_rating = "Menu rating given by the user:\n"
for rating in ratings:
    stringify_rating += f"{rating['variant_id']} : {rating['rating']}\n"

print(stringify_rating)

Menu rating given by the user:
332 : none



In [ ]:
import requests
# Define the API endpoint
url = "http://34.229.85.230:8000/api/request_recommendation/"
# Define the request body
data = {
    "query": "24 years old, 50 kg, 165 cm, lactose intolerant."
}
# Send the POST request
response = requests.post(url,json=data)
# Print the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())  # Assuming the response is in JSON format

Status Code: 200
Response JSON: [{'additional_notes': '24 years old, 50 kg, 165 cm, lactose intolerant.', 'detail_nutritions': ["Based on the user's age, weight, and height, the Harris-Benedict equation suggests a basal metabolic rate (BMR) around 1300 kcal, and considering a sedentary lifestyle, a minimum intake of 1600 kcal is recommended (Mifflin, M. D., St Jeor, S. T., Hill, L. A., Scott, B. J., Daugherty, S. A., & Koh, Y. O. (1990). A new predictive equation for resting energy expenditure in healthy individuals. The American Journal of Clinical Nutrition, 51(2), 241-247).", 'The recommended daily protein intake for adults is 0.8 grams per kilogram of body weight (WHO, 2007).', 'Based on a 24-year-old female, 50 kg, and 165 cm, the minimum fat intake is calculated based on the Dietary Reference Intakes (DRI) which recommends that adults consume 20-35% of their daily calories from fat (Institute of Medicine, 2005).', 'Based on the Dietary Guidelines for Americans, the minimum carboh

In [4]:
import requests
from dataclasses import dataclass
from typing import List

@dataclass
class RatingQuery:
    variant_id: int
    rating: str

@dataclass
class RecommendationQuery:
    gender: str
    age: int
    height: int
    weight: int
    cart_items: List[int]
    bmr_calculation_method: str
    bmr: int
    activity_level: str
    food_preferences: str
    food_allergies: str
    additional_notes: str

def request_recommendation(query: str):
    url = "https://your-api.com/api/request_recommendation/"
    data = {"query": query}
    response = requests.post(url, data=data)
    return response.json()

def request_new_recommendation(query: str, rating: str):
    url = "https://your-api.com/api/get_new_recommendation/"
    data = {"query": query, "rating": rating}
    response = requests.post(url, data=data)
    return response.json()

def build_rating_query(variant_id: int, rating: str) -> RatingQuery:
    return RatingQuery(variant_id=variant_id, rating=rating)

def build_recommendation_query(settings, cart_items, additional_notes: str) -> RecommendationQuery:
    gender = "male" if settings.personal_info.is_male else "female"
    age = settings.personal_info.age
    height = settings.personal_info.height
    weight = settings.personal_info.weight
    
    bmr_calculation_method = {
        "default": "default",
        "custom": "custom",
        "personal_info": "personal_info"
    }.get(settings.bmr_option, "default")
    
    activity_level = {
        "sedentary": "sedentary",
        "light": "light",
        "moderate": "moderate",
        "active": "active",
        "extra_active": "extra active"
    }.get(settings.personal_info.exercise_level, "sedentary")
    
    return RecommendationQuery(
        gender=gender,
        age=age,
        height=height,
        weight=weight,
        cart_items=[item.variant_id for item in cart_items],
        bmr_calculation_method=bmr_calculation_method,
        bmr=settings.custom_bmr_value,
        activity_level=activity_level,
        food_preferences=settings.food_preferences,
        food_allergies=settings.food_allergies,
        additional_notes=additional_notes
    )

# Example data for testing
test_settings = type("Settings", (object,), {
    "personal_info": type("PersonalInfo", (object,), {
        "is_male": True,
        "age": 30,
        "height": 175,
        "weight": 70,
        "exercise_level": "moderate"
    })(),
    "bmr_option": "default",
    "custom_bmr_value": 1800,
    "food_preferences": "vegetarian",
    "food_allergies": "nuts"
})

test_cart_items = [type("CartItem", (object,), {"variant_id": 101})(),
                   type("CartItem", (object,), {"variant_id": 202})()]

test_additional_notes = "Needs high protein diet."

test_recommendation_query = build_recommendation_query(test_settings, test_cart_items, test_additional_notes)

test_rating_query = build_rating_query(variant_id=101, rating="like")

print(test_recommendation_query)
print(test_rating_query)


RecommendationQuery(gender='male', age=30, height=175, weight=70, cart_items=[101, 202], bmr_calculation_method='default', bmr=1800, activity_level='moderate', food_preferences='vegetarian', food_allergies='nuts', additional_notes='Needs high protein diet.')
RatingQuery(variant_id=101, rating='like')


In [23]:
import requests
import json
# Base URL for your local backend server.
BASE_URL = "http://34.229.85.230:8000/"  # Update if necessary
def request_recommendation(recommendation_data):
    """
    Sends a POST request to /api/request_recommendation/ with the recommendation query.
    The recommendation_data is expected to be a dict which will be serialized to JSON.
    """
    url = BASE_URL + "api/request_recommendation/"
    payload = {
        "query": json.dumps(recommendation_data)  # Serialize recommendation query as JSON string.
    }
    response = requests.post(url, data=payload)
    try:
        return response.json()
    except Exception:
        return response.text
def request_new_recommendation(recommendation_data, ratings):
    """
    Sends a POST request to /api/get_new_recommendation/ with the recommendation query and ratings.
    Here, ratings should be a list of rating objects.
    """
    url = BASE_URL + "api/get_new_recommendation/"
    payload = {
        "query": json.dumps(recommendation_data),
        "rating": json.dumps(ratings)  # Serialize the list of rating objects as JSON string.
    }
    response = requests.post(url, data=payload)
    try:
        return response.json()
    except Exception:
        return response.text
def build_sample_recommendation_query():
    """
    Builds a sample recommendation query similar to the Kotlin RecommendationQuery.
    Adjust the values as needed for testing.
    """
    return {
        "gender": "male",                   # or "female" based on your test case.
        "age": 30,                          # sample age.
        "height": 175,                      # sample height in centimeters.
        "weight": 70,                       # sample weight in kilograms.
        "cart_items": [101, 202, 303],        # list of variant IDs in the cart.
        "bmr_calculation_method": "default",# could be "default", "custom", or "personal_info".
        "bmr": 1500,                        # sample BMR value.
        "activity_level": "moderate",       # "sedentary", "light", "moderate", "active", or "extra active".
        "food_preferences": "none",         # sample food preference.
        "food_allergies": "none",           # sample food allergies.
        "additional_notes": "No additional notes."
    }
def build_sample_ratings():
    """
    Builds a sample list of ratings similar to your expected input.
    Each rating object contains a variant_id and a rating string.
    """
    return [
        {"variant_id": 310, "rating": "dislike"},
        {"variant_id": 313, "rating": "none"}
    ]
if __name__ == "__main__":
    # Build sample recommendation query.
    recommendation_query = build_sample_recommendation_query()
    # Test the request_recommendation endpoint.
    print("Testing /api/request_recommendation/ endpoint...")
    rec_response = request_recommendation(recommendation_query)
    print("Response:")
    print(rec_response)
    # Build sample ratings list.
    sample_ratings = build_sample_ratings()
    # Test the request_new_recommendation endpoint.
    print("\nTesting /api/get_new_recommendation/ endpoint...")
    new_rec_response = request_new_recommendation(recommendation_query, sample_ratings)
    print("Response:")
    print(new_rec_response)

Testing /api/request_recommendation/ endpoint...
Response:
{'additional_notes': 'the user has provided the following information about themselves\nthe user has chosen to use the default BMR target of 1500 kcal\nthe user has specified their food preferences as: none\nthe user has listed the following food allergies: none\nthe user has provided the following additional notes: No additional notes.\nthe user has no preference', 'min_nutritions': [1500, 50, 33, 131, 25, 1000, 200], 'recommended_meal_detail': 'Proposed Meal:\n- Homemade Curry (small): A comforting and delicious classic!\n- Miso soup: A light and flavorful soup to complement your meal!\n\nTotal Nutrition:\n- Energy: 541.0 kcal (Target: 500.0 kcal)\n- Protein: 13.6 gr (Target: 16.7 gr)\n- Fat: 14.1 gr (Target: 11.0 gr)\n- Carbohydrate: 85.8 gr (Target: 43.7 gr)\n- Fiber: 3.9 gr (Target: 8.3 gr)\n- Calcium: 28.0 mg (Target: 333.3 mg)\n- Veggies: 56.0 gr (Target: 66.7 gr)', 'list_meals': ['homemade curry (small)', 'miso soup'], 